In [62]:
import re

line = """
https://www.youtube.com/watch?v=wofB1wzyYYI
http://github.com/VikParuchuri/marker/blob/master/tests/renderers/test_markdown_renderer.py
This is a valid text line without a link.
"""
 
url_pattern = re.compile(r"http?s?://[^\s]+")

 
links = []
matches =url_pattern.findall(line)
for match in matches:
 
    links.append({
        "text": match,
        "url": match
    })

for link in links:
    print(link)


{'text': 'https://www.youtube.com/watch?v=wofB1wzyYYI', 'url': 'https://www.youtube.com/watch?v=wofB1wzyYYI'}
{'text': 'http://github.com/VikParuchuri/marker/blob/master/tests/renderers/test_markdown_renderer.py', 'url': 'http://github.com/VikParuchuri/marker/blob/master/tests/renderers/test_markdown_renderer.py'}


In [7]:
import pandas as pd
from pandas import DataFrame
from lxml import html

def extract_tables_from_html(html_content: str) -> list[DataFrame] | None:
    tree = html.fromstring(html_content)
    tables = tree.xpath('//table')

    if not tables:
        return None

    tables_html = [html.tostring(table, method="html").decode() for table in tables]

    print(tables_html)
    dfs:list[DataFrame] = []
    for table_html in tables_html:
        try:
            df = pd.read_html(table_html)[0]
            dfs.append(df)
        except ValueError:
            continue

    return dfs if dfs else None
 
def save_tables_to_markdown(dfs: list[DataFrame], file_name: str) -> None:
    with open(file_name, 'w') as md_file:
        for i, df in enumerate(dfs):
            md_file.write(f"## Table {i + 1}\n\n")
            md_file.write(df.to_markdown())
            md_file.write("\n\n") 
 
with open('sample.html', 'r') as f:
    html_content = f.read()

dfs = extract_tables_from_html(html_content)

if dfs is not None:
    save_tables_to_markdown(dfs, 'tables.md')



In [26]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
import io

def extract_tables_from_html(html_content: str) -> list[DataFrame] | None:
    soup = BeautifulSoup(html_content, "html.parser")
    tables = soup.find_all("table")
    if not tables:
        print("There is no table")
        return None

    tables_html = [str(table) for table in tables]

    dfs:list[DataFrame] = []
    for table_html in tables_html:
        try:
            str_buffer = io.StringIO(table_html)
            df = pd.read_html(str_buffer)[0]
            dfs.append(df)
        except ValueError:
            print("There is no table")
            continue
    if dfs:
        return dfs
    return None

def save_tables_to_markdown(dfs: list[DataFrame], file_name: str) -> None:
    with open(file_name, "w") as md_file:
        for i, df in enumerate(dfs):
            md_file.write(f"## Table {i + 1}\n\n")
            md_file.write(df.to_markdown())
            md_file.write("\n\n")

with open("sample.html") as f:
    html_content = f.read()

dfs = extract_tables_from_html(html_content)
if dfs is not None:
    save_tables_to_markdown(dfs, "tables.md")


yes


In [ ]:
import openpyxl
from openpyxl.drawing.image import Image
from PIL import Image as PILImage

pxl_doc = openpyxl.load_workbook('DimOrder.xlsx')
sheet = pxl_doc['ORDER_DETAILS']
 
for i, image in enumerate(sheet._images):
  
    if isinstance(image, Image):
     
        img_data = image.ref  
    
        pil_img = PILImage.open(img_data)
 
        img_filename = f"image_{i}.png"  
        pil_img.save(img_filename)   
        print(f"Saved image: {img_filename}")


Saved image: image_0.png


In [45]:
sheet_index = ["1", "Order", "3", "5"]

parsed_list = [int(item) if item.isdigit() else item for item in sheet_index]

print(parsed_list)



[1, 'Order', 3, 5]


In [ ]:
import openpyxl
from openpyxl.drawing.image import Image
from PIL import Image as PILImage
 
pxl_doc = openpyxl.load_workbook('DimOrder.xlsx')
 
for sheet_name in pxl_doc.sheetnames:
    sheet = pxl_doc[sheet_name]
 
    for i, image in enumerate(sheet._images):
     
        if isinstance(image, Image):
            img_data = image.ref   
         
            pil_img = PILImage.open(img_data)
 
            img_filename = f"{sheet_name}_image_{i}.png"   
            pil_img.save(img_filename)
            print(f"Saved image from {sheet_name}: {img_filename}")


Saved image from ORDER_DETAILS: ORDER_DETAILS_image_0.png
Saved image from customer: customer_image_0.png
Saved image from vendor: vendor_image_0.png


In [ ]:
print("![](_page_0_Picture_1.png)\n\n# **\"My Agreements\" in 4i**\n\n#### **v8.0| 01/23/2024**\n\nThis Tip Sheet provides information for the Center for Medicare & Medicaid Innovation (CMMI) 4Innovation (4i) application including the use of the \"My Agreements\" functionality.\n\n# **Overview**\n\n4i is a web-based application used by participants and CMMI to manage model operations including entity/practice/provider management, secure data sharing, change request workflows, automation of several business operations, and comprehensive agreement management.\n\nSupport for agreement management in 4i includes the ability to view, electronically sign, download, and track the status of legal agreements. Currently, 4i provides support for models' Participation Agreements and their associated Amendments, Health Insurance Portability and Accountability Act (HIPAA)-Covered Data Disclosure Forms, Data Sharing Agreements, and Novation Agreements.\n\nThis tip sheet provides a general overview of 4i's agreement management functionality, including highlights on accessing, signing, and downloading an entity/practice's agreements.\n\n# **\"My Agreements\"**\n\n4i users can navigate to their agreements by selecting the \"My Agreements\" tab from the leftside navigation pane. You must select the accurate \"Performance Year\" from the drop-down menu located at the top right portion of the \"My Agreements\" page.\n\nTo further filter the page and only display agreements in a specific signing status, you can select the agreement type from the \"Agreement Type\" drop-down (including \"All Agreements,\" \"Participation Agreement,\" \"Participation Agreement Amendment,\" \"Amended & Restated PA,\" and \"HIPAA\") and the applicable status from the \"Signature Status\" (including \"All Signature Status,\" \"Active,\" \"Pending APM Signature,\" \"Pending CMS Signature,\" \"Closed,\" \"Retired,\" \"Expired,\" and \"Terminated\") drop-down.\n\nYou can also access all agreement types in one view for each entity/practice by filtering the page to \"All Agreements\" and \"All Signature Status\" from the \"Agreement Type\" and \"Agreement Status\" drop-downs, respectively. Based on the filtering selections, all of an entity/practice's agreements will be listed in a sorted fashion within the page. The \"Status\" column will only be displayed in the table when the page is filtered by \"All Signature Status.\"\n\nThe \"Related Agreements\" column will contain all the related agreements, regardless of the status for a specific agreement, that are either related to the parent or child agreement. When you select the related agreement hyperlink, you will be navigated to the recital page of the respective agreement. You can either view the recital page of an agreement or download the agreement.\n\n![](_page_1_Picture_1.png)\n\nThe \"My Agreements\" table can be sorted and searched by \"Entity ID\" or \"Practice ID,\" \"Entity Name\" or \"Practice Name\" and \"Agreement ID\" or \"Agreement Name.\" All actions can be completed by selecting the icons in the \"Actions\" column.\n\nPro **Tip:** Hovering over the icons in the \"Actions\" column will display a tooltip with additional details.\n\nPro **Tip:** Hovering over each agreement name will display the full agreement title.\n\n#### **Note:**\n\n- Only Participation Agreements and Amendments that were signed in 4i are accessible in the \"My Agreements\" tab; Participation Agreements and Amendments that were signed outside of 4i can be accessed from the \"Documents\" tab.\n- The ability to sign Participation Agreements and their Amendments is limited to a specified period of time, referred to as the signing window. The dates and times for these windows are not explicitly identified in 4i and will be shared with users outside the system.\n- The icon for signing the Participation Agreement and Amendments will be greyed out for times outside of the specified signing window.\n- The HIPAA-Covered Data Disclosure agreement can be signed at any time as there is not a specified signing window.\n\t- o Please note that PCF Practices will encounter brief periods throughout the year when the HIPAA forms are unavailable to update. For more information, please see the HIPAA Form Availability Schedule in the 4i Knowledge Library.\n\n# **Participation Agreements**\n\nFor most CMMI Models, entities/practices are required to have a bi-laterally signed Participation Agreement in \"Active\" status to participate in the model as part of the onboarding process.\n\n**Note:** Any HIPAA agreement associated with an entity/practice's Participation Agreement can be signed upon creation of the HIPAA agreement even if the entity/practice's Participation Agreement has not been fully executed (counter-signed by CMS). However, there would be no access to PHI/PII protected data until the Participation Agreement is countersigned.\n\n## **Who can sign Participation Agreements in 4i?**\n\nTo sign a Participation Agreement, you must have the role of APM Executive or Practice's Primary Contact. Upon signing your entity/practice's Participation Agreement, the status of the agreement will change to, \"Pending CMS Signature,\" as an indicator that the agreement has been signed by the entity/practice and is now awaiting CMS' review and countersignature. Your Participation Agreement will not be considered \"Active\" or \"Fully Executed\" until CMS has reviewed and countersigned the agreement. Follow each step below to sign the agreement:\n\n![](_page_2_Picture_1.png)\n\n- 1. Once the applicable Performance Year is selected, filter the \"Agreement Type\" to \"Participation Agreement\" and the Signature Status to \"Pending APM Signature\" or \"Pending Practice Signature,\" respectively. Upon filtering the page, the Participation Agreement will display the \"Sign\" (pencil) icon under the \"Actions\" column for each entity or practice with which you are associated.\n- 2. Selecting the enabled \"Sign\" (pencil) icon under the \"Actions\" column will open the Participation Agreement page, displaying the recital, notification, and signature pages from the full Participation Agreement. The \"recital\" provides a high-level summary and the purpose of the agreement in addition to housing the signatures and other information required for the execution of the agreement.\n- 3. After reviewing all details on the page, scroll to the attestation checkbox at the bottom of the displayed agreement to indicate that the APM Executive or Practice Primary Contact is signing the Participation Agreement on the behalf of the entity/practice.\n- 4. Proceed by selecting the \"Sign\" (pencil) icon. After signing, a confirmation message will be displayed on the screen, indicating that you have signed the Participation Agreement on behalf of your entity/practice.\n\n**Pro Tip:** The \"Sign\" (pencil) icon will be enabled when the Participation Agreement is ready to be signed (when the signing window is open) and will be disabled after you have signed the agreement or after the signing window has closed if the entity/practice has not signed.\n\n# **Participation Agreement Amendments**\n\nIf CMS modifies or makes a considerable number of changes to a section of the Participation Agreement for any given model, a Participation Agreement Amendment may be released to those entities/practices currently participating in the model. Once signed and fully executed, the Participation Agreement Amendment, unlike the Amended & Restated Participation Agreement, would not retire the previously signed Participation Agreement(s) by the entity/practice. The Participation Agreement, Participation Agreement Amendment, and associated HIPAA-Covered Data Disclosure Form would remain active.\n\n## **Who can sign Participation Agreement Amendments in 4i?**\n\nTo sign a Participation Agreement Amendment, you must be the entity's APM Executive or Practice's Primary Contact. The agreement signing window must also be open before you can sign. Follow each step below to sign your entity/practice's Participation Agreement Amendment.\n\n- 1. Once the applicable Performance Year is selected, you must filter the \"Agreement Type\" and \"Signature Status\" drop-downs by \"Participation Agreement Amendment\" and \"Pending APM Signature\" or \"Pending Practice Signature,\" respectively.\n- 2. Repeat Steps 2 through 4 above (under Who can sign Participation Agreements in 4i?).\n\n**Note:** A fully executed Participation Agreement Amendment will not retire the previously signed Participation Agreement and any associated agreements. Upon successfully countersignature\n\n![](_page_3_Picture_1.png)\n\nof the Participation Agreement Amendment by CMS, your entity/practice's current Participation Agreement Amendment status will be \"Active\" and any associated agreements will remain active.\n\n# **Amended & Restated Participation Agreement**\n\nIf CMS modifies or makes a considerable number of changes to the Participation Agreement for any given model, an Amended & Restated Participation Agreement may be released to those entities/practices currently participating in the model. Once signed and fully executed, the Amended & Restated Participation Agreement retires any previously signed Participation Agreement(s) by the entity/practice.\n\n### **Who can sign Amended & Restated Participation Agreements in 4i?**\n\nTo sign an Amended & Restated Participation Agreement, you must be the entity's APM Executive or Practice's Primary Contact. The agreement signing window must also be open before you can sign. Follow each step below to sign your entity/practice's Amended & Restated Participation Agreement.\n\n- 1. Once the applicable Performance Year is selected, you must filter the \"Agreement Type\" and \"Signature Status\" drop-downs by \"Amended & Restated Participation Agreement\" and \"Pending APM Signature\" or \"Pending Practice Signature,\" respectively.\n- 2. Repeat Steps 2 through 4 above (under Who can sign Participation Agreements in 4i?).\n\n**Note:** A fully executed Amended & Restated Participation Agreement will retire any previously signed Participation Agreement and any associated agreements, including any Participation Agreement Amendments. This means that upon CMS successfully countersigning the Amended & Restated Participation Agreement, your entity/practice's current Participation Agreement will be moved to a \"Retired\" status, and the newly signed agreement will become \"Active.\"\n\n### **Downloading Participation Agreement(s)/Participation Agreement Amendment(s)/Amended & Restated Participation Agreement(s)**\n\nAll agreements can be downloaded from the \"My Agreements\" page by users with access. The Participation Agreement(s), Participation Agreement Amendment, Amended & Restated Participation Agreement(s), and any associated Amendments can be downloaded after they are fully executed.\n\n**Note:** The \"Download\" icon under the \"Actions\" column will stay disabled until the agreement is fully executed.\n\n#### **Download agreement(s) before signing:**\n\nTo download an agreement before it is fully executed, please follow the steps below.\n\n![](_page_4_Picture_1.png)\n\n- 1. Once the applicable \"Performance Year\" is selected, select the specific agreement type from the \"Agreement Type\" drop-down and then choose the \"Pending APM Signature\" or \"Pending Practice Signature\" from the \"Signature Status\" drop-down to filter the page.\n- 2. Upon filtering the page, the agreement may either display the \"Sign\" (pencil) or \"Review\" (eye) icons under the \"Actions\" column depending on user roles or agreement signature window schedules.\n- 3. If the signing window is open and pending the entity or practice's signature, select either the enabled \"Sign\" (pencil) or the \"Review\" (eye) icons under the \"Actions\" column.\n- 4. Once the recital page loads, you can select the \"Download Full Agreement\" button at the top right corner of the page to download the full version of the agreement.\n- 5. Upon selecting the button, the agreement downloads to your computer as a PDF file that can be viewed or printed when opened.\n\n#### **Download agreement(s) after full execution:**\n\nIf you wish to download an agreement after it is countersigned by CMS, please follow the steps below.\n\n- 1. Once the applicable Performance Year is selected, you must filter the \"Agreement Type\" and \"Signature Status\" drop-downs by specific agreement type and \"Active,\" respectively.\n- 2. Select the \"Download\" (cloud) icon under the \"Actions\" column to download the fully executed agreement as a PDF file. This file will contain a combination of the signed recital page (both CMS and Entity/Practice signatures) and the full version of the agreement.\n\n**Note:** These steps are applicable to all user roles with access to 4i.\n\n#### **Downloading Participation Agreement Recital Page:**\n\nWhen the user selects a Participation Agreement recital page to download in \"My Agreements,\" the following will download as a PDF file in the following order:\n\n- 1. Participation Agreement Recital Page\n- 2. Full Participation Agreement\n- 3. Active PA Amendment(s) Recital Page (at the current point in time or at the point of expiration/termination if the PA is no longer active)\n- 4. Active Full PA Amendment(s)\n\n### **Downloading Active/Retired/Expired PA Amendment Recital Page:**\n\nWhen a user selects an Active/Retired/Expired PA Amendment recital page to download in \"My Agreements,\" the following will download as a PDF file in the following order:\n\n- 1. PA Amendment Recital Page\n![](_page_5_Picture_1.png)\n\n- 2. Full PA Amendment\n#### **Downloading Amended & Restated PA Recital Page:**\n\nWhen the user selects a Participation Agreement Amendment recital page to download in \"My Agreements,\" the following will download as a PDF file in the following order:\n\n- 1. Amended & Restated PA Recital Page\n- 2. Full Amended & Restated PA\n- 3. Active PA Amendment(s) Recital Pages (at the current point in time or at the point of expiration/termination if the A&R PA is no longer active)\n- 4. Active Full PA Amendment(s)\n\n# **Health Insurance Portability and Accountability Act (HIPAA) Covered Data Disclosure Form**\n\n### **What is a HIPAA-Covered Data Disclosure Form?**\n\nThe HIPAA-Covered Data Disclosure Form allows your entity/practice to request certain data from CMS and to attest to your ability to meet the HIPAA requirements for receiving and using such data. This HIPAA Form must be signed by your entity or practice's authorized signatory prior to CMS sharing the data specified in the agreement. The form does not require countersignature from CMS in all models, with the exception of the KCC model, and can be signed prior to the execution of the Participation Agreement. However, there would be no access to PHI/PII protected data until the Participation Agreement is countersigned.\n\n# **Who can sign a HIPAA-Covered Data Disclosure Form in 4i?**\n\nThe entity's APM Executive or Practice Primary Contact is the only user who can complete and sign the form in all models, except for the KCC model. In KCC, upon completion of the required fields and signing of the HIPAA form by the APM Executive, the Health Resources and Services Administration (HRSA) user is required to countersign.\n\n**Note:** The HIPAA-Covered Data Disclosure Form is no longer dependent on an \"Active\" Participation Agreement or Amended & Restated Participation Agreement. This means that the HIPAA form can be signed by a user on behalf of any entity or practice before the Participation Agreement has been fully executed (countersigned by CMS).\n\nPlease follow the steps below to sign the HIPAA Form as the APM Executive/Practice Primary Contact.\n\n- 1. Select \"My Agreements\" on the left navigation pane. Users affiliated with more than one entity or practice will be able to view all entities/practices.\n- 2. Select the \"Performance Year\" drop-down menu at the top right of the page to select the correct Performance Year, if needed.\n\n![](_page_6_Picture_1.png)\n\n- 3. Once the applicable \"Performance Year\" is selected, filter the \"Agreement Type\" and \"Signature Status\" drop-downs to \"HIPAA-Covered Data Disclosure Form\" and \"Pending APM Signature\" or \"Pending Practice Signature,\" respectively.\n- 4. Navigate to the \"Actions\" column for the agreement. The link in the \"Actions\" column will display an enabled \"Sign\" (pencil) icon, indicating that the agreement is ready to be signed. Select the \"Sign\" (pencil) icon to load the agreement.\n\n**Pro Tip:** If the user is the APM Executive/Practice Primary Contact for the entity/practice and the agreement is pending signature, then the \"Actions\" column will display an enabled \"Sign\" (pencil) icon. For all other users and statuses, only the \"Review\" (eye) icon will display.\n\n- 5. Once the agreement loads, make your selections for the questions applicable to your entity/practice. You will also be required to fill out your Data Custodian Contact's details. An Alternate Data Custodian Contact for your entity/practice can optionally be provided on this form.\n- 6. Scroll to the bottom of the page to certify that you have reviewed the form and are ready to sign the agreement. Select the checkbox to activate the \"Sign\" button, then select \"Sign.\"\n\nUpon signing, a success message will be displayed for all models, except for the KCC model, and the status of the HIPAA Form will be updated to \"Active\" if the associated Participation Agreement (Amendment) was countersigned by CMS. Otherwise, the status of the HIPAA Form will be updated to \"Pending Parent Form\" for the KCC Model and \"Pending Parent PA Form\" for the PCF Model and will only update to \"Active\" when the associated Participation Agreement (Amendment) is countersigned by CMS.\n\nIn KCC, upon signing, a success message will display and the status of the HIPAA Form will be updated to \"Pending HRSA Signature.\"\n\nPlease follow the steps below to countersign the HIPAA Form as the HRSA user in KCC.\n\n- 1. Follow Steps 1-2 above.\n- 2. Once the applicable \"Performance Year\" is selected, filter the \"Agreement Type\" and \"Signature Status\" drop-downs to \"HIPAA-Covered Data Disclosure Form\" and \"Pending HRSA Signature\" respectively. Review the details entered by the APM Executive.\n- 3. Navigate to the \"Actions\" column for the agreement. The link in the \"Actions\" column will display an enabled \"Sign\" (pencil) icon, indicating that the agreement is ready to be signed. Select the \"Sign\" (pencil) icon to load the agreement.\n\n**Pro Tip:** If the user is the HRSA user and the agreement is pending HRSA signature, then the \"Actions\" column will display an enabled \"Sign\" (pencil) icon. For all other users and statuses, only the \"Review\" (eye) icon will display.\n\n- 4. Once the agreement loads, scroll to the bottom of the page to certify that you have reviewed the form and are ready to sign the agreement. Select the checkbox to activate the \"Sign\" button, then select \"Sign.\"\n![](_page_7_Picture_1.png)\n\nUpon countersigning, a success message will display and the status of the HIPAA Form will be updated to \"Active\" if the associated Participation Agreement (Amendment) was countersigned by CMS. Otherwise, the status of the HIPAA Form will be updated to \"Pending Parent Form\" and will only update to \"Active\" when the associated Participation Agreement (Amendment) is countersigned by CMS.\n\nTo find the signed form, choose \"HIPAA-Covered Data Disclosure Form\" from the \"Agreement Type\" and \"Active\" from the \"Agreement Status\" drop-down filters.\n\n### **Updating the HIPAA-Covered Data Disclosure Form**\n\nIf the HIPAA-Covered Data Disclosure Form needs to be updated due to changes to your entity's/practice's data custodian details or other information, an updated form must be signed. Please follow the steps below to update your HIPAA form.\n\n- 1. Select \"HIPAA-Covered Data Disclosure Form\" and \"Active\" options from the \"Agreement Type\" and \"Agreement Status\" filters, respectively.\n- 2. Select the Review (eye) icon under the \"Actions\" column.\n- 3. The page will load the previously signed HIPAA form and you can select the \"Update HIPAA Disclosure Form\" button at the top right portion of the page. A pop-up modal containing a warning message will display on the screen to confirm if you would like to proceed with revising your HIPAA-Covered Data Disclosure Form. Selecting \"Yes\" will allow you to update and sign an updated HIPAA form.\n- 4. Follow Steps 5 and 6 above (under Who can sign a HIPAA-Covered Data Disclosure Form in 4i?) to complete and sign the form.\n\n**Note:** You can revise your entity/practice's HIPAA form as many times as you wish. Whenever your entity's HIPAA form is revised, any previously signed HIPAA will be captured under the \"Retired\" category.\n\n### **Downloading the HIPAA-Covered Data Disclosure Form**\n\nTo download a copy of an executed HIPAA form, please follow the steps below.\n\n- 1. Select \"HIPAA Form\" and \"Active\" options from the \"Agreement Type\" and \"Agreement Status\" filters, respectively.\n- 2. Select the \"Download\" (cloud) icon under the \"Actions\" column to download the fully executed agreement as a PDF file.\n\n**Note:** Only versions of agreements that were fully executed in the 4i application will be downloadable (i.e., in \"Active\" status). If a version of your form was migrated from another system, you will get a message that you are unable to download the form.\n\n![](_page_8_Picture_1.png)\n\n# **Novation Agreement**\n\nIf an entity or practice wants to transfer ownership to another corporation, CMS will be notified of this change and a Novation Agreement may be required so that all legal responsibilities and obligations can be successfully transferred. To be considered fully executed, the Novation Agreement will need to be tri-laterally signed by three parties, including the Transferor, Transferee, and CMS.\n\n## **Who can sign the Novation Agreement?**\n\nTo sign a Novation Agreement, you must have the role of APM Executive or Practice Primary Contact. The transferee, if not already in 4i, will need to be invited as a user to complete their required portion of the agreement and can only sign after the Transferor has completed their portion. Upon the Transferee signing the entity/practice's Novation Agreement, the status of the agreement will change to \"Pending CMS Signature.\" This status is an indicator that the agreement has been signed by both preceding parties and is now awaiting CMS' review and countersignature. The Novation Agreement will not be considered \"Active\" or \"Fully Executed\" until CMS has reviewed and countersigned the agreement.\n\n## **Signing the Novation Agreement as a Transferor**\n\nIn all cases, the transferring entity's APM Executive or Practice's Primary Contact may be required to sign the Novation Agreement on behalf of their entity or practice as the Transferor. This indicates that the transferring entity or practice legally agrees to release all rights and responsibilities to the new corporation. To sign the Novation Agreement as the Transferor, follow the steps below:\n\n- 1. Once the applicable Performance Year is selected, filter the \"Agreement Type\" to \"Novation Agreement\" and the \"Signature Status\" to \"Pending APM Signature\" or \"Pending Practice Signature,\" respectively. Upon filtering the page, the Novation Agreement will display the \"Sign\" (pencil) icon under the \"Actions\" column for each entity or practice with which you are associated.\n- 2. Selecting the enabled \"Sign\" (pencil) icon under the \"Actions\" column will open the Novation Agreement page, displaying the recital and signature details.\n- 3. After reviewing all details on the page, scroll to the attestation checkbox at the bottom of the displayed agreement to sign on behalf of the entity/practice as the APM Executive or Practice's Primary Contact.\n- 4. Proceed by selecting the \"Sign\" button at the bottom of the page. After signing, a confirmation message will be displayed on the screen, indicating that you have signed the Novation Agreement on behalf of your entity/practice.\n\n**Note:** The Novation Agreement will remain in \"Pending APM Signature\" or \"Pending Practice Signature\" status after you have signed the Novation Agreement. After the Transferee has signed the agreement, the status will change to \"Pending CMS Signature.\"\n\n![](_page_9_Picture_1.png)\n\n# **Signing the Novation Agreement as a Transferee**\n\nAfter the Transferor has signed the Novation Agreement, the Transferee will be required to sign the Novation Agreement. If the Transferee is not a 4i user, CMS will extend an invitation to the Transferee, which will allow minimal access to 4i after the invitation is accepted. To sign the Novation Agreement as a Transferee, repeat steps 1-4 outlined above (under the Signing the Novation Agreement as a Transferor section). Once the Novation Agreement is countersigned by CMS, all restrictions will be removed and privileges to the application will be elevated (Contact Type will be elevated from Transferee to APM Executive). Access to PHI will be restricted until a new HIPAA-Covered Data Disclosure Form is signed.\n\nSince the Novation Agreement is required to be countersigned before it is considered \"Active,\" the CMS Executive user will be required to complete the signature process.\n\n# **4i's Agreement Signature Status Definitions**\n\n| Signature | Definitions |\n| --- | --- |\n| Status |  |\n| Active | This signature status describes agreements that have completed the |\n|  | execution of its signature workflow. This signature status will be |\n|  | applicable for ALL agreement types. |\n| Pending APM | This signature status describes agreements that require an APM |\n| or Practice | Executive's or Practice Primary Contact's signature. |\n| Signature | Once an agreement is delivered to the model participant, this signature |\n|  | status will indicate an action to be completed by the entity or practice's |\n|  | APM Executive user or Practice Primary contact. |\n| Pending CMS | This signature status describes agreements that require a CMS |\n| Signature | Executive's signature. |\n|  | An agreement type will be considered legally executed and will carry an |\n|  | \"Active\" status, only after CMS has counter-signed the agreement. |\n|  | This status will apply to all agreement types, except the HIPAA Covered |\n|  | Data Disclosure Form as CMS is not required to countersign this |\n|  | agreement. |\n| Closed | This signature status describes agreement types that have never |\n|  | completed the execution of the agreement's signature workflow. These |\n|  | agreements were never active at any point. Example use cases for this |\n|  | status are: |\n|  | i. An entity does not sign any of the agreement types after the |\n|  | signing window has closed. |\n|  | ii. CMS declines to countersign the agreement. |\n\n![](_page_10_Picture_0.png)\n\n**Support** \n\nIf you have questions or require technical assistance, please see the contact information below. **Email:** \n\n- **PCF Model:** PCFTechnicalSupport@cms.hhs.gov\n- **ACO REACH Model:** ACOREACH@cms.hhs.gov\n- **KCC Model:** KCF-CKCC-CMMI@cms.hhs.gov\n\n#### **Phone:**\n\nCall **1-888-734-6433** and then select one of the following:\n\n- **PCF**  option 4\n- **ACO REACH** option 1, and then option 1 again\n- KCC option 1, and then option 3\n\n"
)

![](_page_0_Picture_1.png)

# **"My Agreements" in 4i**

#### **v8.0| 01/23/2024**

This Tip Sheet provides information for the Center for Medicare & Medicaid Innovation (CMMI) 4Innovation (4i) application including the use of the "My Agreements" functionality.

# **Overview**

4i is a web-based application used by participants and CMMI to manage model operations including entity/practice/provider management, secure data sharing, change request workflows, automation of several business operations, and comprehensive agreement management.

Support for agreement management in 4i includes the ability to view, electronically sign, download, and track the status of legal agreements. Currently, 4i provides support for models' Participation Agreements and their associated Amendments, Health Insurance Portability and Accountability Act (HIPAA)-Covered Data Disclosure Forms, Data Sharing Agreements, and Novation Agreements.

This tip sheet provides a general overview of 4i's agreement management